In [1]:
import pymongo as pmg
import pandas as pd
import datetime as dt
import pymysql
import matplotlib.pyplot as plt
from matplotlib.lines import TICKLEFT, TICKRIGHT, Line2D
from matplotlib.patches import Rectangle
import matplotlib as mpl
import matplotlib.dates as mdate
from datetime import time
from KRData.HKData import HKFuture
import configparser
import talib as tb

In [2]:
conf_parser = configparser.ConfigParser()
conf_parser.read( 'conf.ini')
mg_host = conf_parser.get('MGDB', 'host')
mg_user = conf_parser.get('MGDB', 'user')
mg_password = conf_parser.get('MGDB', 'password')
hk=HKFuture(pwd=mg_password,host=mg_host,user=mg_user)
begin='2014-01-01'
dt1=dt.datetime.now()
_fields = ['datetime', 'code', 'open', 'high', 'low', 'close', 'vol']
df1=hk.get_main_contract_bars('HSI', _fields, begin)
print(dt.datetime.now()-dt1)

0:00:59.535022


In [2]:
#df1.to_pickle("pkl/hsi.pkl")
df1=pd.read_pickle("pkl/hsi.pkl")

In [3]:
dt1=dt.datetime.now()
df1['MACD'],df1['MACD1'],df1['MACD2']=tb.MACD(df1.close.values,fastperiod=6,slowperiod=12,signalperiod=9)
df1['MA30']=tb.EMA(df1.close.values,timeperiod=30)
df1['MA60']=tb.EMA(df1.close.values,timeperiod=60)
df1['CHG']=df1['close']-df1['open']
df1['STD60']=tb.STDDEV(df1['CHG'].values,timeperiod=60)
df1['STD1']=df1['CHG']/df1['STD60']
print(dt.datetime.now()-dt1)

0:00:00.359714


In [46]:
time1=dt.datetime.now()
ref_MACD=0
MACD_In=0
MA60_In=0
Dict_MACD={}
Dict_MA60={}
Res_MACD=[]
Res_MA60=[]
Dict_MACD['State']=0
Dict_MACD['StateE']=0
MACD_NO=0
MACD_NO1=0
MACD_NO2=0
Dict_MA60['State']=0
ref_Close=0
dt1=dt.time(9,15,0)
dt2=dt.time(16,30,0)
cont={}
for i,row in df1.iterrows():
    #State 1>ma60,2<ma6 MACD红|3>ma60,4<ma60 MACD 绿区
    #持续MACD红区
    cont['MACD1']=row["MACD"]>0 and Dict_MACD['State'] in [1,2]
    #持续MACD绿区
    cont['MACD2']=row['MACD']<0 and Dict_MACD['State'] in [3,4]
    #转MACD红区
    cont['MACD3']=row['MACD']>0 and Dict_MACD['State'] in [0,3,4]
    #l转MACD绿区
    cont['MACD4']=row['MACD']<0 and Dict_MACD['State'] in [0,1,2]
    
#     print(row["MACD"],cont)
    #持续MA60红区
    cont['MA60a']=row["close"]>row['MA60'] and Dict_MA60['State']==1
    #持续MA60绿区
    cont['MA60b']=row['close']<row['MA60'] and Dict_MA60['State']==-1
    #转MA60红区
    cont['MA60c']=row['close']>=row['MA60'] and Dict_MA60['State']<=0
    #l转MA60绿区
    cont['MA60d']=row['close']<row['MA60'] and Dict_MA60['State']>=0
    
    #MA60计算
    if cont['MA60c'] or cont['MA60d']:
        #rec
        if MA60_In>0:
            Dict_MA60['Close']=row['MA60']
            Dict_MA60['End']=i
            if i.time()>dt2 or i.time()<dt1:
                Dict_MA60['Day']=0
            else:
                Dict_MA60['Day']=1
            Res_MA60.append(Dict_MA60)
            Dict_MA60={}
        Dict_MA60['Index']=MA60_In
        Dict_MA60['Begin']=i
        Dict_MA60['Open']=row['close']
        Dict_MA60['High']=row['close']
        Dict_MA60['Low']=row['close']
        Dict_MA60['Count']=1
        Dict_MA60['Std1']=1 if row['STD1']>=1.5 else 0
        Dict_MA60['Std2']=1 if row['STD1']<=-1.5 else 0
        MA60_In+=1
        Dict_MA60['State']=1 if cont['MA60c'] else -1
        MACD_NO=0
        MACD_NO1=0
        MACD_NO2=0
    elif cont['MA60a'] or cont['MA60b']:
        Dict_MA60['Count']+=1
        
        if row['STD1']>=1.5:
            Dict_MA60['Std1']+=1
        elif row['STD1']<=-1.5:
            Dict_MA60['Std2']+=1
            
        if row['close']>Dict_MA60['High']:
            Dict_MA60['High']=row['close']
        elif row['close']<Dict_MA60['Low']:
            Dict_MA60['Low']=row['close']
    
    #MACD计算
    if cont['MACD3'] or cont['MACD4']:
        #rec
        #Type 1 B&E>0 3 B>0&E<0 2 B&E<0 4 B<0&E>0
        if MACD_In>0:
            Dict_MACD['Close']=row['close']
            Dict_MACD['End']=i
#             print(Dict_MACD['Type'],row["close"],row["MA60"])
            if row["MACD"]>0 and row["close"]>row['MA60']:
                MACD_NO2+=1
                Dict_MACD['No']=MACD_NO2
                Dict_MACD['StateE']=1
            elif row["MACD"]>0 and row["close"]<=row['MA60']:
                MACD_NO2+=-1
                Dict_MACD['No']=MACD_NO2
                Dict_MACD['StateE']=2
            elif row["MACD"]<=0 and row["close"]>row['MA60']:
                MACD_NO1+=1
                Dict_MACD['No']=MACD_NO1
                Dict_MACD['StateE']=3
            elif row["MACD"]<=0 and row["close"]<=row['MA60']:
                MACD_NO1+=-1
                Dict_MACD['No']=MACD_NO1
                Dict_MACD['StateE']=4
            else:
                Dict_MACD['State']=0
                
            if i.time()>dt2 or i.time()<dt1:
                Dict_MACD['Day']=0
            else:
                Dict_MACD['Day']=1
            Res_MACD.append(Dict_MACD)
            Dict_MACD={}
#             Dict_MACD['Type']=0
            
        Dict_MACD['Index']=MACD_In
        Dict_MACD['Begin']=i
        Dict_MACD['Open']=row['close']
        Dict_MACD['High']=row['close']
        Dict_MACD['Low']=row['close']
        Dict_MACD['Count']=1
        Dict_MACD['Std1']=1 if row['STD1']>=1.5 else 0
        Dict_MACD['Std2']=1 if row['STD1']<=-1.5 else 0
        MACD_In+=1
#         print(MACD_In)
        if row["MACD"]>0 and row["close"]>row['MA60']:
            Dict_MACD['State']=1
        elif row["MACD"]>0 and row["close"]<=row['MA60']:
            Dict_MACD['State']=2
        elif row["MACD"]<=0 and row["close"]>row['MA60']:
            Dict_MACD['State']=3
        elif row["MACD"]<=0 and row["close"]<=row['MA60']:
            Dict_MACD['State']=4
        else:
            Dict_MACD['State']=0
        Dict_MACD['Type']=1 if row["close"]>row['MA60'] else 2
    elif cont['MACD1'] or cont['MACD2']:
        Dict_MACD['Count']+=1
        ref_Close=row['close']
        
        if row['STD1']>=1.5:
            Dict_MACD['Std1']+=1
        elif row['STD1']<=-1.5:
            Dict_MACD['Std2']+=1
            
        if row['close']>Dict_MACD['High']:
            Dict_MACD['High']=row['close']
        elif row['close']<Dict_MACD['Low']:
            Dict_MACD['Low']=row['close']
time2=dt.datetime.now()
print(time2-time1)

0:02:40.948456


In [26]:
Ma60=pd.DataFrame(Res_MA60)
Ma60['Diff']=Ma60['Close']-Ma60['Open']
Ma60.head()

,Begin,Close,Count,Day,End,High,Index,Low,Open,State,Std1,Std2,Diff
0,2014-01-02 10:14:00,23236.056955,89,1,2014-01-02 11:43:00,23350.0,0,23184.0,23299.0,-1,2,6,-62.943045
1,2014-01-02 11:43:00,23235.825580,1,1,2014-01-02 11:44:00,23243.0,1,23243.0,23243.0,1,0,0,-7.174420
2,2014-01-02 11:44:00,23235.280723,3,1,2014-01-02 11:47:00,23229.0,2,23227.0,23229.0,-1,0,0,6.280723
3,2014-01-02 11:47:00,23257.329920,26,1,2014-01-02 13:13:00,23293.0,3,23236.0,23236.0,1,2,0,21.329920
4,2014-01-02 13:13:00,23257.483037,1,1,2014-01-02 13:14:00,23255.0,4,23255.0,23255.0,-1,0,1,2.483037


In [34]:
Calc=['sum','max','min','mean','std','count']
# Ma60[(Ma60.State==1) & (Ma60.Day==0)].groupby('Std1')['Diff'].agg(Calc)
Ma60[Ma60.Day==1].groupby('State')['Diff'].agg(Calc)

,sum,max,min,mean,std,count
State,,,,,,
-1,-114882.090881,303.318723,-1385.882749,-6.110424,44.461035,18801
1,114962.007296,966.291834,-223.709106,6.106555,39.984369,18826


In [47]:
MACD=pd.DataFrame(Res_MACD)
MACD['Diff']=MACD['Close']-MACD['Open']
MACD.head(5)

,Begin,Close,Count,Day,End,High,Index,Low,No,Open,State,StateE,Std1,Std2,Type,Diff
0,2014-01-02 09:34:00,23439.0,1,1,2014-01-02 09:35:00,23466.0,0,23466.0,NaN,23466.0,0,0.0,0,0,2,-27.0
1,2014-01-02 09:35:00,23430.0,1,1,2014-01-02 09:36:00,23439.0,1,23439.0,NaN,23439.0,0,NaN,0,0,2,-9.0
2,2014-01-02 09:36:00,23411.0,1,1,2014-01-02 09:37:00,23430.0,2,23430.0,NaN,23430.0,0,NaN,0,0,2,-19.0
3,2014-01-02 09:37:00,23416.0,1,1,2014-01-02 09:38:00,23411.0,3,23411.0,NaN,23411.0,0,NaN,0,0,2,5.0
4,2014-01-02 09:38:00,23406.0,1,1,2014-01-02 09:39:00,23416.0,4,23416.0,NaN,23416.0,0,NaN,0,0,2,-10.0


In [48]:
Calc=['sum','max','min','mean','std','count']
MACD[(MACD.State==1) & (MACD.Day==1)].groupby('Std1')['Diff'].agg(Calc)
MACD[(MACD.State==-1)].groupby('No')['Diff'].agg(Calc)
# MACD[(MACD.State==-1) & (MACD.Day==1)].groupby('Std2')['Diff'].agg(Calc)
MACD.groupby(['State'])['Diff'].agg(Calc)

,sum,max,min,mean,std,count
State,,,,,,
0,-167.0,18.0,-38.0,-4.175000,11.788494,40
1,3276.0,589.0,-566.0,0.144228,35.717800,22714
2,-13900.0,893.0,-992.0,-1.043309,38.221335,13323
3,14938.0,820.0,-775.0,1.012403,33.798218,14755
4,1488.0,594.0,-810.0,0.069918,39.349497,21282


In [49]:
MACD.groupby(['StateE'])['Diff'].agg(Calc)

,sum,max,min,mean,std,count
StateE,,,,,,
0.0,-27.0,-27.0,-27.0,-27.000000,NaN,1
1.0,251814.0,820.0,-143.0,11.086290,18.864577,22714
2.0,-235388.0,133.0,-810.0,-17.667793,51.100154,13323
3.0,238725.0,893.0,-235.0,16.179261,46.651799,14755
4.0,-249360.0,116.0,-992.0,-11.716393,21.193059,21283


In [31]:
df2[(df2.State==1) & (df2.Day==1)]['Diff'].agg(['max','min','mean','std'])

max     893.000000
min    -992.000000
mean     -0.072913
std      45.811485
Name: Diff, dtype: float64

In [32]:
df2[df2.State==1]['Count'].agg(['max','min','mean','std'])

max     157.00000
min       1.00000
mean     13.26239
std      13.48442
Name: Count, dtype: float64

In [44]:
df2['hh']=df2.apply(lambda x:x.Begin.time().hour,axis=1)
df2[df2.State==1].groupby('hh')['Diff'].agg(['max','min','mean','std','count'])

,max,min,mean,std,count
hh,,,,,
0,589.0,-992.0,-1.061135,89.104121,687
9,882.0,-235.0,0.349847,59.895631,1958
10,489.0,-105.0,0.440603,40.222055,2921
11,457.0,-451.0,0.082313,35.316139,2940
12,0.0,-142.0,-38.200000,58.942345,5
13,498.0,-82.0,-1.740180,30.936376,3106
14,439.0,-108.0,-0.388438,31.849561,2958
15,380.0,-440.0,0.187910,38.575829,2895
16,329.0,-566.0,1.058598,41.184382,1041


In [24]:
time1=dt.time(9,15,0)
time2=dt.time(16,30,0)
newDF=df1[(df1.datetime.dt.time>=time1)&(df1.datetime.dt.time<=time2)]
newDF.head(10)

,datetime,code,open,high,low,close,MACD,MACD1,MACD2,MA30,MA60,CHG,STD60,STD1
datetime,,,,,,,,,,,,,,
2014-01-02 09:15:00,2014-01-02 09:15:00,HSI1401,23414.0,23416.0,23370.0,23376.0,NaN,NaN,NaN,NaN,NaN,-38.0,NaN,NaN
2014-01-02 09:16:00,2014-01-02 09:16:00,HSI1401,23376.0,23394.0,23360.0,23394.0,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN
2014-01-02 09:17:00,2014-01-02 09:17:00,HSI1401,23395.0,23444.0,23391.0,23431.0,NaN,NaN,NaN,NaN,NaN,36.0,NaN,NaN
2014-01-02 09:18:00,2014-01-02 09:18:00,HSI1401,23432.0,23435.0,23419.0,23423.0,NaN,NaN,NaN,NaN,NaN,-9.0,NaN,NaN
2014-01-02 09:19:00,2014-01-02 09:19:00,HSI1401,23426.0,23426.0,23390.0,23401.0,NaN,NaN,NaN,NaN,NaN,-25.0,NaN,NaN
2014-01-02 09:20:00,2014-01-02 09:20:00,HSI1401,23401.0,23405.0,23384.0,23403.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
2014-01-02 09:21:00,2014-01-02 09:21:00,HSI1401,23403.0,23418.0,23401.0,23415.0,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN
2014-01-02 09:22:00,2014-01-02 09:22:00,HSI1401,23415.0,23430.0,23415.0,23430.0,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN
2014-01-02 09:23:00,2014-01-02 09:23:00,HSI1401,23430.0,23497.0,23430.0,23456.0,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN


In [96]:
def Calc(Date):
# Date='2014-01-06'
    D1=newDF[str(Date)]
    STD=1.5
    Res={}
    Res['STD_CNT']=0
    Res['State']=0
    Res['Trade']=[]
    Trade={}
    Trade['Open']=[]
    for Row in D1.iterrows():
        Rec=Row[1]

        if Rec['MACD']<0 and Res['State']==2:
            Trade['Date']=Date
            Trade['Close']=Rec['close']
            Trade['Profit']=[Rec['close']-i for i in Trade['Open']]
            Trade['Profits']=sum(Trade['Profit'])
            Trade['Count']=len(Trade['Profit'])
            Res['Trade'].append(Trade)
#             print(Trade)
            Res['State']=0
            Res['STD_CNT']=0
            Trade={}
            Trade['Open']=[]
        elif Rec['MACD']<0:
            Res['State']=0
            Res['STD_CNT']=0
            Trade={}
            Trade['Open']=[]
        elif Res['State']==0:
            Res['State']=1
        if Rec['STD1']>STD and Res['State']>0:
            Res['STD_CNT']+=1
            if Res['STD_CNT']>=3:
                Trade['Open'].append(Rec['close'])
    #             print(Row[0],Res['STD_CNT'],Rec['close'])
                Res['State']=2
    return Res['Trade']

In [101]:
dl=newDF.datetime.dt.date.unique()
#固定止损
Data=[]
time1=dt.datetime.now()
for dd in dl:
    get=Calc(dd)
    if len(get)>0:
        for i in get:
            Data.append(i)
time2=dt.datetime.now()
print(time2-time1)

0:00:34.545976


In [115]:
F=pd.DataFrame(Data)
# Data
F.head()

,Close,Count,Date,Open,Profit,Profits
0,23354.0,5,2014-01-02,"[23292.0, 23317.0, 23327.0, 23357.0, 23377.0]","[62.0, 37.0, 27.0, -3.0, -23.0]",100.0
1,22654.0,1,2014-01-06,[22669.0],[-15.0],-15.0
2,22726.0,8,2014-01-06,"[22654.0, 22666.0, 22682.0, 22690.0, 22712.0, ...","[72.0, 60.0, 44.0, 36.0, 14.0, -2.0, -6.0, -12.0]",206.0
3,22689.0,1,2014-01-07,[22702.0],[-13.0],-13.0
4,22758.0,5,2014-01-08,"[22780.0, 22786.0, 22798.0, 22782.0, 22793.0]","[-22.0, -28.0, -40.0, -24.0, -35.0]",-149.0


In [4]:
a=3
a in[3,4]

True